# AOP CAM Demo, March 17, 2020

Introductory text.

## Shared code

Each request will need to be encoded as a JSON object and sent to our API.

In [4]:
import requests
import json

def getRequest(query):
    endpoint = "http://robokop.renci.org:6434/query"
    r = requests.post(endpoint, json=query)
    result = r.json()
    resultGraph = result["knowledge_graph"]
    return resultGraph

We also need to convert our graph format into one that is understandable by the Cytoscape visualization library we use.

In [39]:
from cyjupyter import Cytoscape

def reasonerGraphToCytoscape(graph):
    csGraph = {}
    nodes = []
    edges = []
    for node in graph["nodes"]:
        csNode = {}
        csNode["data"] = {"id": node["id"], "label": node.get("label", node.get("id"))}
        nodes.append(csNode)
    for edge in graph["edges"]:
        csEdge = {
            "data": {
                "label": edge["id"],
                "id": edge["id"],
                "source": edge["source_id"],
                "target": edge["target_id"]
            }
        }
        edges.append(csEdge)
    csGraph["elements"] = {"nodes": nodes, "edges": edges}
    csGraph["style"] = [
        {
            "selector": "node",
            "style": {
                "label": "data(label)",
                "color": "white",
                "background-color": "#60f",
                "shape": "rectangle",
                "text-valign": "center",
                "width": "20em",
                "height": "5em",
                "text-wrap": "wrap"
            }
        },
        {
            "selector": "edge",
            "style": {
                "curve-style": "unbundled-bezier",
                "control-point-distances": [
                    -20,
                    20
                ],
                "control-point-weights": [
                    0.5
                ],
                "content": "data(label)",
                "line-color": "#808080",
                "target-arrow-color": "#808080",
                "target-arrow-shape": "triangle",
                "target-arrow-fill": "filled"
            }
        }
    ]
                       
    # print(json.dumps(csGraph, indent=4))
    return csGraph

# Example query 1

In this query, we look for biological processes that have AXL receptor tyrosine kinase ([NCBIGENE:558](https://www.ncbi.nlm.nih.gov/gene/558)) as a participant.

In [40]:
query = json.loads("""
{
  "message": {
    "query_graph": {
      "nodes": [
        {
          "id": "n0",
          "type": "gene",
          "curie": "NCBIGENE:558",
          "label": "NCBIGENE:558"
        },
        {
          "id": "n1",
          "type": "biological_process",
          "label": "biological_process"
        }
      ],
      "edges": [
        {
          "id": "e0",
          "source_id": "n1",
          "target_id": "n0",
          "type": "has_participant",
          "label": "has_participant"
        }
      ]
    }
  }
}
""")

We can visualize this data.

In [41]:
queryData = reasonerGraphToCytoscape(query["message"]["query_graph"])
Cytoscape(data=queryData, visual_style=queryData["style"], layout_name="breadthfirst", layout={"height": "700px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'NCBIGENE:558'}}, {'data': {'id': 'n1', …

We can query our service with this query, and examine the results.

In [42]:
resultGraph = getRequest(query)
print(json.dumps(resultGraph, indent=4))
Cytoscape(data=reasonerGraphToCytoscape(resultGraph), visual_style=queryData["style"], layout_name='breadthfirst', layout={"height": "700px"})

{
    "nodes": [
        {
            "id": "obo:BFO_0000002",
            "name": null,
            "type": []
        },
        {
            "id": "GO:0010467",
            "name": "gene expression",
            "type": [
                "biological_process",
                "biological_process_or_activity",
                "occurrent"
            ]
        },
        {
            "id": "NCBIGENE:558",
            "name": "AXL",
            "type": [
                "macromolecular_machine",
                "gene",
                "gene_or_gene_product"
            ]
        },
        {
            "id": "GO:0016310",
            "name": "phosphorylation",
            "type": [
                "biological_process",
                "biological_process_or_activity",
                "occurrent"
            ]
        },
        {
            "id": "GO:0032259",
            "name": "methylation",
            "type": [
                "biological_process",
                "biological_

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'obo:BFO_0000002', 'label': 'obo:BFO_0000002'}}, {'data…